In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data_folder = "./output/no_lowdin/device"
self_energy = np.load(f"{data_folder}/self_energy.npy", allow_pickle=True)
nodes = np.load(f"{data_folder}/nodes.npy")
index_active_region = np.load(f"{data_folder}/index_active_region.npy")
print("self_energy_shape",self_energy[0].shape)
print("nodes",nodes)
print("index active region", index_active_region)


self_energy_shape (648, 648)
nodes [    0  3888  6736 10624]
index active region [    3    12    21 ... 10600 10609 10618]


In [41]:
def expand_sparse_self_energy(sigma_small, index_active, target_size, target_range):
    """
    Expand a sparse self-energy matrix to a dense matrix of size (target_size, target_size),
    inserting values only at the positions that lie within target_range.

    Parameters
    ----------
    sigma_small : np.ndarray
        (n_active, n_active) self-energy matrix in the active subspace.
    index_active : np.ndarray
        Indices in the full space where the active orbitals reside.
    target_size : int
        Size of the expanded self-energy matrix (typically a lead block size).
    target_range : tuple
        (start, end) range in full space corresponding to the lead region.

    Returns
    -------
    sigma_expanded : np.ndarray
        (target_size, target_size) full-sized self-energy with values inserted at appropriate indices.
    """
    start, end = target_range
    print(start,end)
    sigma_expanded = np.zeros((target_size, target_size), dtype=complex)

    # Map global indices to local within the lead block
    local_indices = np.array([i - start for i in index_active if start <= i < end])
    active_indices = np.array([i for i in index_active if start <= i < end])
    print(local_indices)
    print(active_indices)
    if len(local_indices) == 0:
        raise ValueError("No active orbitals found in the target range.")

    for i_new, i_old in enumerate(active_indices):
        for j_new, j_old in enumerate(active_indices):
            sigma_expanded[local_indices[i_new], local_indices[j_new]] = sigma_small[i_old, j_old]

    return sigma_expanded


In [46]:
sigma_left = expand_sparse_self_energy(
    sigma_small=self_energy[0].Sigma,
    index_active=index_active_region,
    target_size=nodes[1] - nodes[0],
    target_range=(nodes[0], nodes[1])
)

sigma_right = expand_sparse_self_energy(
    sigma_small=self_energy[1].Sigma,
    index_active=index_active_region,
    target_size=nodes[3] - nodes[2],
    target_range=(nodes[2], nodes[3])
)


ValueError: Inconsistent shape: sigma_small is (648, 648) but 432 active orbitals found in lead range.